In [ ]:
## TAAS 교통사고데이터 좌표 크롤러 with Selenium
#Selenium을 이용한 동적 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

C:\Users\sons\AppData\Local\Temp\ipykernel_26760\3160177533.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [39]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [42]:
#1. 융합분석(고속도로별) 위도,경도 크롤링 시도 -> 실패


#융합분석(menuRoadNoSearch) 선택
menuRoadNoSearch_element = driver.find_element(By.ID, 'menuRoadNoSearch')

# menuRoadNoSearch를 찾았다면 클릭
if menuRoadNoSearch_element:
    menuRoadNoSearch_element.click()


# selectBox지정
#2018년 시작
selectboxStart = driver.find_element(By.ID, 'ptsNafYearStart')
select = Select(selectboxStart)
select.select_by_value("2018") 

#2020년 종료
selectboxYearEnd = driver.find_element(By.ID, 'ptsNafYearEnd')
select = Select(selectboxYearEnd)
select.select_by_value("2020") 

#고속국도 조회
selectboxRoadRank = driver.find_element(By.ID, 'ptsNafRoadRank')
select = Select(selectboxRoadRank)
select.select_by_visible_text("고속국도") 

#도로명 지정
selectboxRoadName = driver.find_element(By.ID, 'ptsNafCh1RoadName')
select = Select(selectboxRoadName)
select.select_by_visible_text("구마고속도로") 



#checkbox 지정
#조건설정 전부 체크
checkbox_elements = driver.find_elements(By.CSS_SELECTOR, '#ptsNafCh1AccidentContent input[type="checkbox"]')

# Check all checkboxes
for checkbox in checkbox_elements:
    if not checkbox.is_selected():
        checkbox.click()



In [43]:
# search
driver.find_element(By.XPATH, '//*[@id="roadNumberAccidentFind"]/div/p/a').send_keys(Keys.ENTER)
time.sleep(1)


In [49]:
coord_ls = []
srs_ls = []

#결과 개수만큼 반복문
for i in tqdm(range(0, 1)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);')  # 개별 사고지점
    time.sleep(1)  # 로드

    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-2]
    # 디버깅 코드
    if isinstance(bbox, list):
        print("Debugging - bbox content:")
        for element in bbox:
            print(element)
    else:
        print("Debugging - bbox content:", bbox)

    src = parse.unquote(bbox.get_attribute('src'))  # get src and unquote from 16byte

    # 디버깅 코드
    if isinstance(src, list):
        print("Debugging - src content:")
        for element in src:
            print(element)
    else:
        print("Debugging - src content:", src)

    src_split = src.split('&')
    print("src_split : " + str(src_split))  # Use str() to convert list to string

    if isinstance(src_split, list):
        print("Debugging - src_split:")
        for element in src_split:
            print(element)
    else:
        print("Debugging - src_split:", src_split)

    bbox_text = src_split[-3][5:]
    print("bbox_text : " + bbox_text)
    # coords = [float(i) for i in bbox_text.split(',')]
    # x, y = (coords[0] + coords[2]) / 2, (coords[1] + coords[3]) / 2  # center position
    # srs = src_split[-4][4:]
    # coord_ls.append([x, y])
    # srs_ls.append(srs)


  0%|          | 0/1 [00:01<?, ?it/s]

Debugging - bbox content: <selenium.webdriver.remote.webelement.WebElement (session="d63eb421735be5afad0ac026cc4e64d9", element="BE8747241E6CA8D597D51FF1DD7A5961_element_7496")>
Debugging - src content: https://map3.daumcdn.net/map_2d/2212ejo/L13/-2/6.png
src_split : ['https://map3.daumcdn.net/map_2d/2212ejo/L13/-2/6.png']
Debugging - src_split:
https://map3.daumcdn.net/map_2d/2212ejo/L13/-2/6.png


IndexError: list index out of range